# 네이버뉴스 크롤링


- 날짜, 언론사, 제목, 카테고리, url, 내용

- 댓글 크롤링

In [46]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os
import pickle as pk
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm

#크롤링
from bs4 import BeautifulSoup
import requests
import re
import sys
import csv, json


### Date and Author: 20230731, Kyungwon Kim ###
### 네이버뉴스를 크롤링할 url 생성 및 뉴스 추출 함수 만들기
headers = {
    'authority': 'apis.naver.com',
    'accept': '*/*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'ab.storage.userId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%22browser-1625985144309-6%22%2C%22c%22%3A1626101500089%2C%22l%22%3A1626101500089%7D; ab.storage.deviceId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%224cbe130c-6edd-d4aa-a78d-290b003c3592%22%2C%22c%22%3A1626101500094%2C%22l%22%3A1626101500094%7D; ASID=7992e0220000017aaa36664e0000004e; _ga=GA1.2.612969395.1626832328; ab.storage.sessionId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%2228148006-e01d-7623-b7d1-b4fff0f59b4e%22%2C%22e%22%3A1627919390179%2C%22c%22%3A1627908091281%2C%22l%22%3A1627917590179%7D; MM_NEW=1; NFS=2; NNB=RDIIILNX6JCWE; nx_ssl=2; nid_inf=1665554565; NID_AUT=tP3V5ox533EjyAgkJ1JaqWEnPOhXs2hr3teD39pK972fuXqDWQZXoIOMzICJpa1A; NID_JKL=d393brIzilbjw+7TVvG0OW6Eo22+WIhQAfihItUdgbY=; _naver_usersession_=SPdJTrlTMrn8Udkyn58eo6HL; NID_SES=AAABwJaKJ5FjUAETXL8SAX2HKMUSTt3l8pPu49OSzbGzgKEEMN/ckpP4DbQVHQwTV1hVPWtbpP7Nomg0CbD8TtCpyOYbeq8+OpHb5eWbDsXXCeLHO4epgthLtbQHiBE8spXqEtx/h0D6MzxsIlN4pa8gz51jV+oWzQQNnpQCeaKKLaxcpMfhGXnZv4BK1Rg+TAgUFE9RtExcKjteTL2hB9tKT41C7antdQdhLfVXWUbsJ/q5b62iDZnnZUAANXHnWp/9RI2YyKSn70SVu4Bag+fxA/23OqjCHSbK5RMiNOQKV+Bs7uugaAsMKkH6lGBBIbNDkTXGZ4n1+KbqFwe1kV9oCaPJ+siwXESEqvY0jaLVNAqUATQZjnIMFIYwARw41FTuduxW1IOF7MdP7R3EqOvnqNir2lXW1UfRlHlOtMC4w/tXk8xqJR/HVlZrnltKkMZB5zfyDNvnt02jbOKJcORjmOeVvL+xoCdSXwZclfJzRkC31l43+9jSu4X8RPUfuJILRMHf2e1A0NU7Mwds7h+S//5AD0yUJlPtFFzLvriuD1SMTRXiSwN4pNWBi6UIsPzScRpyLMc8hUE8Bi8jJtGk4e0=; NDARK=N; page_uid=hrKUflprvN8ssNc4Muwssssss3R-382317; BMR=',
    'referer': 'https://n.news.naver.com/article/028/0002595736',
    'sec-ch-ua': '"Whale";v="3", " Not;A Brand";v="99", "Chromium";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Whale/3.15.136.18 Safari/537.36',
}


### 날짜 설정에 따른 URL 추출
def get_urls_navernews_bydate(search_query, start_date, end_date, sort=0, maxpage=1000):
    start = datetime.datetime.now()
    i, urls = 1, []
    while True:
        page = str((i-1) * 10 + 1)
        url = f"https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={search_query}&sort={sort}&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date}&start={page}"
        urls.append(url)
        if i == maxpage+1:
            end = datetime.datetime.now()
            print('URL Extracting Time: ', end-start)
            return urls
        else:
            i = i + 1
### URL에 담긴 댓글 추출
def get_navernews_comments(url):
    # setting
    url = url.split('?')[0]
    oid_1, oid_2 = url.split('/')[-1], url.split('/')[-2]

    i, comments = 1, [] #모든 댓글을 담는 리스트
    while True:
        params = {
            'ticket': 'news',
            'templateId': 'default_society',
            'pool': 'cbox5',
            'lang': 'ko',
            'country': 'KR',
            'objectId': f'news{oid_2},{oid_1}',
            'pageSize': '100',
            'indexSize': '10',
            'page': str(i),
            'currentPage': '0',
            'moreParam.direction': 'next',
            'moreParam.prev': '10000o90000op06guicil48ars',
            'moreParam.next': '1000050000305guog893h1re',
            'followSize': '100',
            'includeAllStatus': 'true',
        }

        response = requests.get('https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json', params=params, headers=headers)
        response.encoding = "UTF-8-sig"
        res = response.text.replace("_callback(","")[:-2]
        temp=json.loads(res)
        try :
            comment = list(pd.DataFrame(temp['result']['commentList'])['contents'])
            for j in range(len(comment)):
                comments.append(comment[j])
            else:
                i+=1
        except :
            break

    return comments

### URL에 담긴 댓글을 포함한 뉴스정보 추출
def get_navernews(url):
    start = datetime.datetime.now()
    time_articles, press_articles, category_articles, title_articles, content_articles, comment_articles = [], [], [], [], [], []
    url_articles, url_articles_naver = [], []
    for pg in tqdm(url):
        response = requests.get(pg, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 페이지 내 기사시간, 언론사, 제목, 기사URL 불러오기
        news_elements = soup.select('div.news_wrap.api_ani_send')
        for element in news_elements:
            time = element.select_one('span.info').text.strip()
            time_articles.append(time)
            press = element.select_one('a.info.press').text.strip()
            press_articles.append(press)
            title = element.select_one('a.news_tit').text.strip()
            title = re.sub(pattern='<[^>]*>', repl='', string=str(title))
            title_articles.append(title)
            each_url = element.select_one('a.news_tit')['href']
            url_articles.append(each_url)
            

        # 기사URL 중 네이버뉴스 주소로 반영된 것은 업데이트
        article_elements = soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
        for article in article_elements:
            ## 네이버 URL이 없는 경우는 빈칸 저장
            if "news.naver.com" not in article.attrs['href']:
                url_articles_naver.append(article.attrs['href'])
                category_articles.append([])
                content_articles.append([])
                comment_articles.append([])
            ## 네이버 URL이 있는 경우 내용 저장
            else:   
                # 링킹
                url_articles_naver[-1] = article.attrs['href'] 
                article_response = requests.get(url_articles_naver[-1], headers=headers, verify=False)
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                # 카테고리 불러오기
                category = article_soup.select_one('#_LNB > ul > li.Nlist_item._LNB_ITEM.is_active > a > span')
                if category != None:
                    category_articles[-1].append(str(category).split('menu">')[1].split('</span>')[0])
                else:
                    category_articles[-1].append([])
                
                # 본문 불러오기 "div#dic_area"에서 "article#dic_area"로 변경
                content = article_soup.select("article#dic_area")
                if content == []:
                    content = article_soup.select("#articeBody")
                ## 본문 전처리 정리
                content = ''.join(str(content))
                content = re.sub(pattern='<[^>]*>', repl='', string=content)
                content = content.replace("""[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}""", '')
                content = content.replace('\n', ' ').replace('\t', ' ')
                content_articles[-1] = content
                # 댓글 불러오기
                comment = get_navernews_comments(url_articles_naver[-1])
                comment_articles[-1] = comment         
                # 기사시간 업데이트
                try:
                    time_html = article_soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
                    time = time_html.attrs['data-date-time']
                    time_articles[len(comment_articles)-1] = time
                except AttributeError:
                    time = article_soup.select_one("#content > div.end_ct > div > div.article_info > span > em")
                    time = re.sub(pattern='<[^>]*>',repl='',string=str(time))
                    time_articles[len(comment_articles)-1] = time
        
        # 마지막 페이지면 종료
        if len(news_elements) < 10: 
            break

            # 정리
    df_news = pd.DataFrame({'Date':time_articles,
                            'Press':press_articles,
                            'Category':category_articles,
                            'Title':title_articles,
                            'Content':content_articles,
                            'Comment':comment_articles,
                            'URL_Origin':url_articles,
                            'URL_Naver':url_articles_naver})
    
    end = datetime.datetime.now()
    print('News Info Extracting Time: ', end-start)
    print('Size of News Data: ', df_news.shape[0])
    
    return df_news

In [47]:
# search_query = "세대 갈등"
# sort = 0
# start_page = 1
    # end_page = 5
#
# url = get_urls_navernews_bypage(search_query, start_page, end_page, sort)
# news = get_navernews(url)

search_query = "세대 갈등"
sort = 0
start_date = "20230601"
end_date = "20230631"

url = get_urls_navernews_bydate(search_query, start_date, end_date, sort, maxpage=2)
news = get_navernews(url)

URL Extracting Time:  0:00:00


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.65s/it]

News Info Extracting Time:  0:00:07.965958
Size of News Data:  30


In [48]:
news

,Date,Press,Category,Title,Content,Comment,URL_Origin,URL_Naver
0,2023-06-29 00:26:01,한국경제언론사 선정,[오피니언],[고승연의 세대공감] 이젠 MZ도 아닌 잘파세대?…갈등은 그대로 둔채 세대론만...,[ 지난 수년간 ‘MZ세대론’ 열풍이 몰아치다 드디어 좀 사그라드는가 싶더니...,"[세대론은 끝없이 나올 겁니다. "" 요즘 것들은 말이야 "" 를 다르게 말한게 MZ세...",https://www.hankyung.com/opinion/article/20230...,https://n.news.naver.com/mnews/article/015/000...
1,2023.06.27.,기호일보,[],어르신 소통방법 알고 나니 세대갈등 없어요,[],[],http://www.kihoilbo.co.kr/news/articleView.htm...,http://www.kihoilbo.co.kr/
2,2023.06.23. 오전 9:18,엑스포츠뉴스,[[]],"'지락실2', 또 양심고백→세대갈등까지…""사기꾼들 아니에요?""","[ (엑스포츠뉴스 이예진 기자) ‘언니즈’ 이은지와 미미, ‘동생즈’ 이영지와 ...",[],https://www.xportsnews.com/article/1739411,https://n.news.naver.com/mnews/article/311/000...
3,2023.06.11.,청년일보,[],"[청년발언대] MZ세대, 세대갈등의 불쏘시개가 되다",[],[],https://www.youthdaily.co.kr/news/article.html...,http://www.youthdaily.co.kr/
4,2023-06-21 04:09:11,국민일보언론사 선정,[경제],청년특공에 중장년은 “역차별”… ‘로또’ 공공분양 세대갈등 양상,[ 수도권서 시세의 70∼80% 수준尹공약 ‘군필자 가점’ 도입 주장도국토부는 “제...,"[분명히 역차별 맞다. 청년세대는 자라나는 세대이고, 나라의 미래를 책임지는 세대...",https://news.kmib.co.kr/article/view.asp?arcid...,https://n.news.naver.com/mnews/article/005/000...
5,2023-06-29 17:25:15,매일경제,[오피니언],[매경춘추] 세대 갈등,[ 세면대 옆 딸아이의 화장품을 보고 깜짝 놀란다. '다 컸구나'라는 생각도...,[],https://www.mk.co.kr/article/10772542,https://n.news.naver.com/mnews/article/009/000...
6,2023.06.14.,미주중앙일보,[],[독자 마당] 디지털 시대 세대갈등,[],[],https://news.koreadaily.com/2023/06/13/society...,http://koreadaily.com
7,2023-06-09 14:23:01,JTBC언론사 선정,[정치],"국민의힘 태영호 후임 최고위원에 김가람…""세대 갈등 잇는 역할 하고파""",[ 국민의힘 최고위원 보궐선거에서 김가람 후보가 당선됐습니다. 국민의힘은 ...,"[인물 드럽게 없는갑다!, 침몰 만이 답이다. 국짐들아 지금 잘하고 있다. 이대로 ...",https://news.jtbc.co.kr/article/article.aspx?n...,https://n.news.naver.com/mnews/article/437/000...
8,2023-06-08 15:43:07,한겨레,[생활/문화],"EBS, 세대·시대 갈등 담긴 ‘저출산’ 문제 해법 찾는다",[ ‘다큐멘터리K-인구대기획 초저출생’ 14일 시작 아빠의 육아휴직 등 돌볼 ...,[저출산 컨셉 그만 울거 먹어라 지겹다. 이제 틀렸다. 그냥 인구 천만 국가에 대비...,https://www.hani.co.kr/arti/culture/culture_ge...,https://n.news.naver.com/mnews/article/028/000...
9,2023.06.20.,이뉴스투데이,[],"""청년 혜택, 이번엔 과하다""‧‧‧세대갈등 경계론 고개 들다",[],[],http://www.enewstoday.co.kr/news/articleView.h...,http://www.enewstoday.co.kr/


In [49]:
df = pd.DataFrame(news)

In [52]:
import pandas as pd
from datetime import datetime

# 데이터프레임을 생성하거나 불러오는 코드 (이미 있는 df를 사용하고자 한다면 해당 코드를 사용하세요)

# 시간 형식 변환 함수
def convert_time_format(input_str):
    try:
        if '오전' in input_str:
            input_str = input_str.replace('오전', 'AM')
        elif '오후' in input_str:
            input_str = input_str.replace('오후', 'PM')
        
        # 날짜 및 시간 형식 변환
        dt = datetime.strptime(input_str, '%Y.%m.%d. %p %I:%M')
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        return input_str  # 날짜 형식이 아닌 경우 그대로 반환

# 'Date' 컬럼을 변환
df['Date'] = df['Date'].apply(convert_time_format)

# 네이버뉴스만 남기기
df = df[df['URL_Naver'].str.contains('https://n.news.naver.com/mnews/article/')]

# 결과 확인
print(df)

                   Date       Press Category  \
0   2023-06-29 00:26:01  한국경제언론사 선정   [오피니언]   
2   2023-06-23 09:18:00      엑스포츠뉴스     [[]]   
4   2023-06-21 04:09:11  국민일보언론사 선정     [경제]   
5   2023-06-29 17:25:15        매일경제   [오피니언]   
7   2023-06-09 14:23:01  JTBC언론사 선정     [정치]   
8   2023-06-08 15:43:07         한겨레  [생활/문화]   
10  2023-06-05 12:10:15        문화일보     [사회]   
19  2023-06-13 16:01:01  한국일보언론사 선정     [사회]   
20  2023-06-30 05:58:00         뉴스엔     [[]]   
21  2023-06-21 17:52:00       오마이뉴스     [[]]   
27  2023-06-09 15:18:58   머니S언론사 선정     [정치]   

                                              Title  \
0    [고승연의 세대공감] 이젠 MZ도 아닌 잘파세대?…갈등은 그대로 둔채 세대론만...   
2                '지락실2', 또 양심고백→세대갈등까지…"사기꾼들 아니에요?"   
4               청년특공에 중장년은 “역차별”… ‘로또’ 공공분양 세대갈등 양상   
5                                      [매경춘추] 세대 갈등   
7           국민의힘 태영호 후임 최고위원에 김가람…"세대 갈등 잇는 역할 하고파"   
8                  EBS, 세대·시대 갈등 담긴 ‘저출산’ 문제 해법 찾는다   
10          13년뒤 청년 2명이 노인 1명 부양… 늘어나는

In [53]:
# 날짜형식이 24시간 포맷으로 업데이트 됨
df

,Date,Press,Category,Title,Content,Comment,URL_Origin,URL_Naver
0,2023-06-29 00:26:01,한국경제언론사 선정,[오피니언],[고승연의 세대공감] 이젠 MZ도 아닌 잘파세대?…갈등은 그대로 둔채 세대론만...,[ 지난 수년간 ‘MZ세대론’ 열풍이 몰아치다 드디어 좀 사그라드는가 싶더니...,"[세대론은 끝없이 나올 겁니다. "" 요즘 것들은 말이야 "" 를 다르게 말한게 MZ세...",https://www.hankyung.com/opinion/article/20230...,https://n.news.naver.com/mnews/article/015/000...
2,2023-06-23 09:18:00,엑스포츠뉴스,[[]],"'지락실2', 또 양심고백→세대갈등까지…""사기꾼들 아니에요?""","[ (엑스포츠뉴스 이예진 기자) ‘언니즈’ 이은지와 미미, ‘동생즈’ 이영지와 ...",[],https://www.xportsnews.com/article/1739411,https://n.news.naver.com/mnews/article/311/000...
4,2023-06-21 04:09:11,국민일보언론사 선정,[경제],청년특공에 중장년은 “역차별”… ‘로또’ 공공분양 세대갈등 양상,[ 수도권서 시세의 70∼80% 수준尹공약 ‘군필자 가점’ 도입 주장도국토부는 “제...,"[분명히 역차별 맞다. 청년세대는 자라나는 세대이고, 나라의 미래를 책임지는 세대...",https://news.kmib.co.kr/article/view.asp?arcid...,https://n.news.naver.com/mnews/article/005/000...
5,2023-06-29 17:25:15,매일경제,[오피니언],[매경춘추] 세대 갈등,[ 세면대 옆 딸아이의 화장품을 보고 깜짝 놀란다. '다 컸구나'라는 생각도...,[],https://www.mk.co.kr/article/10772542,https://n.news.naver.com/mnews/article/009/000...
7,2023-06-09 14:23:01,JTBC언론사 선정,[정치],"국민의힘 태영호 후임 최고위원에 김가람…""세대 갈등 잇는 역할 하고파""",[ 국민의힘 최고위원 보궐선거에서 김가람 후보가 당선됐습니다. 국민의힘은 ...,"[인물 드럽게 없는갑다!, 침몰 만이 답이다. 국짐들아 지금 잘하고 있다. 이대로 ...",https://news.jtbc.co.kr/article/article.aspx?n...,https://n.news.naver.com/mnews/article/437/000...
8,2023-06-08 15:43:07,한겨레,[생활/문화],"EBS, 세대·시대 갈등 담긴 ‘저출산’ 문제 해법 찾는다",[ ‘다큐멘터리K-인구대기획 초저출생’ 14일 시작 아빠의 육아휴직 등 돌볼 ...,[저출산 컨셉 그만 울거 먹어라 지겹다. 이제 틀렸다. 그냥 인구 천만 국가에 대비...,https://www.hani.co.kr/arti/culture/culture_ge...,https://n.news.naver.com/mnews/article/028/000...
10,2023-06-05 12:10:15,문화일보,[사회],13년뒤 청년 2명이 노인 1명 부양… 늘어나는 부담에 세대갈등 현실로,[ ■ 지역소멸극복 현장을 가다 - (1) 함양군의 ‘인생 이모작 프로젝트’...,[],https://www.munhwa.com/news/view.html?no=20230...,https://n.news.naver.com/mnews/article/021/000...
19,2023-06-13 16:01:01,한국일보언론사 선정,[사회],페미는 거른다 vs 반페미는 아웃… 절반세대 결혼관에서 젠더는 '거름망',[ [절반 쇼크가 온다: 1-③절반세대 연애·결혼·출산 리포트]인식조사에서 여실히 ...,[페미니즘 가지고 남성혐오를 하는걸 본 세대인데 자신이 페미라는 사람과 결혼 하길 ...,https://www.hankookilbo.com/News/Read/A2023060...,https://n.news.naver.com/mnews/article/469/000...
20,2023-06-30 05:58:00,뉴스엔,[[]],"MZ세대 딘딘, 정치인들에 작심 발언 “공과 사 구별할 것” (관출금)[결정적장...",[ [뉴스엔 장예솔 기자] '관계자 외 출입금지' 딘딘이 현직 국회의원들에게 조...,[],https://www.newsen.com/news_view.php?uid=20230...,https://n.news.naver.com/mnews/article/609/000...
21,2023-06-21 17:52:00,오마이뉴스,[[]],MZ세대 취향 가득한데... 긴 러닝타임이 문제네,[ [리뷰] 영화 &lt;스파이더맨: 어크로스 더 유니버스&gt; ▲ 영화...,[],https://www.ohmynews.com/NWS_Web/View/at_pg.as...,https://n.news.naver.com/mnews/article/047/000...


In [54]:
df.to_csv('navernews_june', index=False)

In [58]:
df.to_csv('navernews_june.csv', encoding='utf-8-sig',index=False)